# Explore Twitter news comments
Let's look at the comments on news articles posted on Twitter to determine how representative they are of different audiences.

In [4]:
import pandas as pd
auth_data = pd.read_csv('../../data/auth_data/twitter_auth.csv', sep=',', index_col=False).iloc[0, :]

In [68]:
import twitter
twitter_api = twitter.Api(consumer_key=auth_data.loc['consumer_api_key'],
                          consumer_secret=auth_data.loc['consumer_api_key_secret'],
                          access_token_key=auth_data.loc['auth_access_token'],
                          access_token_secret=auth_data.loc['auth_access_token_secret'])

In [12]:
test_twitter_account = 'nytimes'
test_twitter_timeline = twitter_api.GetUserTimeline(screen_name=test_twitter_account)
print(test_twitter_timeline)

[Status(ID=1358850596200136706, ScreenName=nytimes, Created=Mon Feb 08 18:50:03 +0000 2021, Text='The discovery of a mysterious metal slab in Turkey was a throwback to November, when a shiny monolith appeared in t… https://t.co/nJRrRJjFNZ'), Status(ID=1358849355030142977, ScreenName=nytimes, Created=Mon Feb 08 18:45:07 +0000 2021, Text='Dogecoin, a digital currency created as a joke, is now the world’s 10th largest cryptocurrency. Its price is up by… https://t.co/eKc609Dqpd'), Status(ID=1358848086748053505, ScreenName=nytimes, Created=Mon Feb 08 18:40:04 +0000 2021, Text='Three climbers who set out to scale the summit of K2, the world’s second-tallest mountain, have been missing since… https://t.co/liVhNrIzt1'), Status(ID=1358847932854919168, ScreenName=nytimes, Created=Mon Feb 08 18:39:28 +0000 2021, Text='Rescue efforts were underway on Monday in Uttarakhand, where a Himalayan glacier broke and caused massive flooding.… https://t.co/eWFl2c9SB1'), Status(ID=1358846839076839426, Screen

Let's look for posts within a small time frame.

In [71]:
start_date_str

'2018-04-01'

In [74]:
# help(twitter_api.GetSearch)
from datetime import datetime
test_twitter_account = 'nytimes'
start_date = (2018, 4, 1)
end_date = (2018, 5, 1)
start_date_str = '%d-%.2d-%.2d'%start_date
end_date_str = '%d-%.2d-%.2d'%end_date
## TODO: how to get tweets from time interval from specific account?
# test_raw_query = f'q=from:({test_twitter_account}) until:{end_date_str} since:{start_date_str})'
test_raw_query = f'q=from%3A{test_twitter_account} until%3A{end_date_str}'
# start_time = datetime(*start_date)
# end_time = datetime(*end_date)
# test_posts = twitter_api.GetSearch(term='@nytimes')
test_posts = twitter_api.GetSearch(raw_query=test_raw_query)

TwitterError: [{'code': 32, 'message': 'Could not authenticate you.'}]

Update: to do the full archive search we need to write the query from scratch? Cool.

Following this recipe: https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/master/Full-Archive-Search/full-archive-search.py

In [95]:
import requests
from datetime import datetime
import json
from time import sleep
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers
SLEEP_TIME=60*5
def connect_to_endpoint(url, headers, params):
    success = False
    while(not success):
        response = requests.request("GET", search_url, headers=headers, params=params)
    #     print(response.status_code)
        # rate-limit => sleep to recover
        if(response.status_code == 429):
            print(f'sleeping for {SLEEP_TIME} because rate limit error {response}')
            sleep(SLEEP_TIME)
        elif(response.status_code != 200):
            raise Exception(response.status_code, response.text)
        else:
            success = True
    return response.json()

def collect_all_tweets(search_url, headers, query_params):
    combined_tweets = []
    has_next_page = True
    ctr = 0
    while(has_next_page):
        json_response = connect_to_endpoint(search_url, headers, query_params)
        if('data' in json_response):
            combined_tweets.extend(json_response['data'])
        has_next_page = ('meta' in json_response) and ('next_token' in json_response['meta'])
        if(has_next_page):
            query_params['next_token'] = json_response['meta']['next_token']
        ctr += 1
        if(ctr % 10 == 0):
            print(f'collected {len(combined_tweets)} total')
    return combined_tweets
    
bearer_token = auth_data.loc['bearer_token']
search_url = "https://api.twitter.com/2/tweets/search/all"
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
user_account = 'nytimes'
start_date = datetime(year=2018, month=4, day=1)
end_date = datetime(year=2018, month=5, day=1)

time_zone = '-05:00'
date_fmt = f'%Y-%m-%dT%H:%M:%S{time_zone}'
query_params = {'query': f'(from:{user_account})', 
                'start_time' : f'{datetime.strftime(start_date, date_fmt)}', 
                'end_time':f'{datetime.strftime(end_date, date_fmt)}',
                'max_results' : 500,
                'tweet.fields' : ['id', 'text'],
#                 'media.fields' : ['url'] # TODO: get full URL
               }
headers = create_headers(bearer_token)
query_tweets = collect_all_tweets(search_url, headers, query_params)
print(json.dumps(query_tweets))
# print(len(json_response))
# print(json.dumps(json_response, indent=4, sort_keys=True))

sleeping for 300 because rate limit error <Response [429]>
[{"id": "991179622137163776", "text": "The new CDC director's $375,000 salary is getting cut https://t.co/8VobyiPlRe"}, {"id": "991175159829680129", "text": "President Trump was long advised to avoid tweeting about Stormy Daniels. But he did, and now she's suing for defamation. https://t.co/5HoElKHf7W"}, {"id": "991170606719791104", "text": "RT @nytimesworld: Israel's Netanyahu says stolen Iranian documents prove Iran had a secret nuclear weapons program, but cites no evidence t\u2026"}, {"id": "991165837896798209", "text": "Detroit was crumbling. Here\u2019s how it\u2019s reviving. https://t.co/dnMPpUyJqp https://t.co/ddgE8gKfZc"}, {"id": "991162058338193408", "text": "RT @noamscheiber: HUGE gig economy news today: California Supreme Court takes a sledgehammer to the gig economy business model. Easily the\u2026"}, {"id": "991158296508854272", "text": "A T-Mobile-Sprint merger could affect your cellphone bill, whether you're th

In [98]:
import pandas as pd
pd.DataFrame(query_tweets[:10])

,id,text
0,991179622137163776,"The new CDC director's $375,000 salary is gett..."
1,991175159829680129,President Trump was long advised to avoid twee...
2,991170606719791104,RT @nytimesworld: Israel's Netanyahu says stol...
3,991165837896798209,Detroit was crumbling. Here’s how it’s revivin...
4,991162058338193408,RT @noamscheiber: HUGE gig economy news today:...
5,991158296508854272,A T-Mobile-Sprint merger could affect your cel...
6,991156012165095424,Today was the deadliest day for journalists in...
7,991154522432122881,RT @nytopinion: .@PaulKrugman writes that it n...
8,991150730651340801,"Roy Moore sued 4 of his accusers, alleging tha..."
9,991146946676051969,RT @grynbaum: CBS News execs considered withdr...


In [99]:
## write to file
import pandas as pd
out_file = '../../data/twitter_data/sample_nytimes_data.gz'
query_tweet_data = pd.DataFrame(query_tweets)
query_tweet_data.to_csv(out_file, sep='\t', compression='gzip', index=False)

In [110]:
query_tweet_data.loc[:, 'text'].iloc[0]

"The new CDC director's $375,000 salary is getting cut https://t.co/8VobyiPlRe"

In [131]:
## expand URLs in tweet
import http.client
# def expandURL(link):
#     conn = http.client.HTTPSConnection(link) #use HTTPS !
#     conn.request('HEAD')
#     response = conn.getresponse()
#     return response.getheader('location')
# import urllib3
test_url = "https://t.co/8VobyiPlRe"
import requests
def expand_url(link):
    response = requests.get(link)
    full_url = response.url
    return full_url
test_full_url = expand_url(test_url)
print(test_full_url)

https://www.nytimes.com/2018/04/30/health/cdc-director-salary-redfield.html?smid=tw-nytimes&smtyp=cur


In [139]:
## expand all URLs
import re
url_matcher = re.compile('https://t\.co[^…#]+$') # ignore ellipsis, hashtags
query_tweet_data = query_tweet_data.assign(**{
    'post_URL' : query_tweet_data.loc[:, 'text'].apply(lambda x: url_matcher.findall(x) if url_matcher.search(x) is not None else None)
})
url_query_tweet_data = query_tweet_data[query_tweet_data.loc[:, 'post_URL'].apply(lambda x: x is not None)]
from time import sleep
SLEEP_TIME = 1 # need sleep time to avoid IP getting blocked lol
def retrieve_urls(links):
    full_URLs = []
    for link in links:
        full_URL = expand_url(link)
        full_URLs.append(full_URL)
        sleep(SLEEP_TIME)
    return full_URLs
from tqdm import tqdm
tqdm.pandas()
url_query_tweet_data = url_query_tweet_data.assign(**{
    'full_URLs' : url_query_tweet_data.loc[:, 'post_URL'].progress_apply(lambda x: retrieve_urls(x))
})
## cleanup URL end text

100%|██████████| 2930/2930 [1:36:58<00:00,  1.99s/it]


In [145]:
import re
url_suffix_matcher = re.compile('(?<=html)\?.+$')
# test_url = 'https://www.nytimes.com/2018/04/30/us/politics/questions-mueller-wants-to-ask-trump-russia.html?partner=rss&emc=rss&smid=tw-nytimes&smtyp=cur'
# url_suffix_matcher.sub('', test_url)
url_query_tweet_data = url_query_tweet_data.assign(**{
    'clean_URLs' : url_query_tweet_data.loc[:, 'full_URLs'].apply(lambda x: list(map(lambda y: url_suffix_matcher.sub('', y), x)))
})

In [147]:
url_query_tweet_data.loc[:, 'clean_URLs'].iloc[0]

['https://www.nytimes.com/2018/04/30/health/cdc-director-salary-redfield.html']

How many of these tweets overlap with the articles that we already have?

In [100]:
news_article_data = pd.read_csv('../../data/nyt_comments/ArticlesApril2018.csv', sep=',', index_col=False)
display(news_article_data.head())

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [151]:
news_article_URLs = set(news_article_data.loc[:, 'webURL'].unique())
url_query_tweet_data = url_query_tweet_data.assign(**{
    'overlap_URLs' : url_query_tweet_data.loc[:, 'clean_URLs'].apply(lambda x: list(set(x) & news_article_URLs))
})

In [153]:
overlap_news_tweet_articles = set([y for x in url_query_tweet_data.loc[:, 'overlap_URLs'].values for y in x])
print(f'{len(overlap_news_tweet_articles)}/{len(news_article_URLs)} articles mentioned by at least one tweet')

556/1324 articles mentioned by at least one tweet


OK! We can recover about 42% of all observed articles from the Twitter data, which is pretty good considering the delay between data collection and data generation.

Let's save this data for later.

In [154]:
url_query_tweet_data.to_csv('../../data/twitter_data/sample_nytimes_URL_data.gz', sep='\t', compression='gzip', index=False)

### Assess engagement via replies

Now that we have a reasonable amount of posts, let's see if we can get reply-tweets.

Key question: do we have a reasonable diversity of authors in terms of {geography, age, experience}?